# Performance metrics

In [2]:
!curl https://wagon-public-datasets.s3.amazonaws.com/insurance_ML3.csv > ../data/insurance_ML3.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 68717  100 68717    0     0   146k      0 --:--:-- --:--:-- --:--:--  147k


In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('../data/insurance_ML3.csv')

In [5]:
data.head()

,age,sex,bmi,children,smoker,region,charges,price_range
0,19,female,27.900,0,True,southwest,16884.92400,expensive
1,18,male,33.770,1,False,southeast,1725.55230,cheap
2,28,male,33.000,3,False,southeast,4449.46200,cheap
3,33,male,22.705,0,False,northwest,21984.47061,expensive
4,32,male,28.880,0,False,northwest,3866.85520,cheap


Taking the `mean` as a baseline. We use `DummyRegressor` for that.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

# Prepare X and y
X = data[['age','bmi','children','smoker']]
y = data['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 6) # Holdout

baseline_model = DummyRegressor(strategy="mean") # Baseline
baseline_model.fit(X_train, y_train) # Calculate value for strategy
baseline_model.score(X_test, y_test) # Score model based on consistently predicting the strategy

-0.001233635021205659

`LinearRegression` socre improves a lot the baseline score

In [8]:
model = LinearRegression().fit(X_train, y_train) # instantiate and fit model 

model.score(X_test, y_test) # Score model

0.773425820295562

## Comparing metrics

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
import math

mse = mean_squared_error(y, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y, y_pred)
rsquared = r2_score(y, y_pred)
max_error = max_error(y, y_pred)

print('MSE =', round(mse, 2))
print('RMSE =', round(rmse, 2))
print('MAE =', round(mae, 2))
print('R2 =', round(rsquared, 2))
print('Max Error =', round(max_error, 2))

### Cross-validation

We can specify the scoring metric in cross-validation

In [10]:
from sklearn.model_selection import cross_validate
# 5-Fold Cross validate model
model = LinearRegression()
cv_results = cross_validate(model, X, y, cv=5, scoring=['max_error', 'r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']
                           )
pd.DataFrame(cv_results) # Cross validation output

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error
0,0.009710,0.004429,-24053.301788,0.760959,-4210.447467,-3.714635e+07
1,0.003643,0.003108,-23060.600919,0.708823,-4219.303638,-3.781926e+07
2,0.003098,0.003147,-26395.712047,0.776167,-4026.911043,-3.285122e+07
3,0.003293,0.002785,-23282.860901,0.731409,-4297.280431,-3.987217e+07
4,0.003397,0.002007,-29700.376643,0.756647,-4218.673537,-3.696269e+07


### Confusion matrix

In [12]:
y_test = [0, 1, 0, 0, 1, 0, 1, 1, 0, 1] # actual truths
preds = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1] # predictions

results_df = pd.DataFrame({"actual": y_test,
                           "predicted": preds}) #Store results in a dataframe

confusion_matrix = pd.crosstab(index= results_df['actual'],
                               columns = results_df['predicted'])
confusion_matrix

predicted,0,1
actual,,
0,3,2
1,1,4


## Comparing Metrics

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = [0, 1, 0, 0, 1, 0, 1, 1, 0, 1]
y_pred = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

print('Accuracy =', round(accuracy_score(y_true, y_pred), 4)) # Accuracy

print('Precision =', round(precision_score(y_true, y_pred), 4)) # Precision

print('Recall =', round(recall_score(y_true, y_pred), 4)) # Recall

print('F1 score =', round(f1_score(y_true, y_pred), 4)) # F1 score

Accuracy = 0.7
Precision = 0.6667
Recall = 0.8
F1 score = 0.7273


### Precision recall curve

In [16]:
data.head()

,age,sex,bmi,children,smoker,region,charges,price_range
0,19,female,27.900,0,True,southwest,16884.92400,expensive
1,18,male,33.770,1,False,southeast,1725.55230,cheap
2,28,male,33.000,3,False,southeast,4449.46200,cheap
3,33,male,22.705,0,False,northwest,21984.47061,expensive
4,32,male,28.880,0,False,northwest,3866.85520,cheap


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

model = LogisticRegression()

# Predict class probabilties
data['prob_cheap'], data['prob_expensive'] =cross_val_predict(model, X, data['price_range_encoded'], cv=5, method='predict_proba').T

# precision recall data
precision, recall, threshold = precision_recall_curve(data['price_range_encoded'], data['proba_expensive']) 

print(f'precision- {precision[:5]}')
print(f'recall- {recall[:5]}')
print(f'threshold- {threshold[:5]}')

In [ ]:
# Find threshold that guarantees 0.8 recall score
scores[scores['recall'] >= 0.8].threshold.max()

In [ ]:
model = LogisticRegression()
model.fit(X, data['price_range_encoded'])

def custom_predict(X, custom_threshold):
    probs = model.predict_proba(X) # Get likelihood of each sample being classified as 0 or 1
    expensive_probs = probs[:, 1] # Only keep expensive likelihoods (1) 
    return (expensive_probs > custom_threshold).astype(int) # Boolean outcome converted to 0 or 1 
    
updated_preds = custom_predict(X=X, custom_threshold=0.305539) # Update predictions 

print(recall_score(data['price_range_encoded'], updated_preds)) # Rerun recall
print(precision_score(data['price_range_encoded'], updated_preds)) # Rerun precision
print(f1_score(data['price_range_encoded'], updated_preds)) # Rerun f1

### ROC-AUC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Extract associated metrics and thresholds
fpr, tpr, thresholds = roc_curve(data['price_range_encoded'], data['proba_expensive'])

# Compute AUC score
auc_score = roc_auc_score(y_true, y_pred)
auc_score